# JIGSAW DEMONSTRATION
Demonstrates Jigsaw working, and breaking Jigsaw through it's renormalisation

In [2]:
import qiskit
import numpy as np
import copy
import PatchedMeasCal.jigsaw as jigsaw
from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal import bv, qft
from PatchedMeasCal.fake_backends import Grid, Hexagonal, Hexagonal16
from PatchedMeasCal.utils import norm_results_dict

from PatchedMeasCal.state_prep_circuits import integer_state_prep_sim, integer_state_dist

from PatchedMeasCal.gen_error_matrices import FakeMeasurementError

from PatchedMeasCal.fake_backends import LocalSimulator


import qiskit.tools.jupyter

In [3]:
n_qubits = 4
n_shots = 16000
bv_str = '0' * (n_qubits - 2) + '11'
targ_bv_str = bv_str[::-1]

backend = Grid(2, 2) #Hexagonal16() #Grid(3, 3)

circuit = bv.bv_circuit(bv_str, n_qubits)
print(targ_bv_str)
backend

1100


<Grid('square')>

## Bare execution

In [4]:
initial_layout = list(range(n_qubits))

tc = qiskit.transpile(circuit, backend=backend, optimization_level=0, initial_layout=initial_layout)
res_d = qiskit.execute(tc, backend, shots=n_shots, optimization_level=0, initial_layout=initial_layout).result().get_counts()
res_bare = copy.deepcopy(res_d)
norm_results_dict(res_d)
print(res_d[targ_bv_str])

0.8586875


## Run Jigsaw

In [5]:
res_j = jigsaw.jigsaw(circuit, backend, n_shots, verbose=True, local_pmf_pairs = [(0, 3), (1, 2)])
print(res_j[targ_bv_str])

0.9883356984008368


## Try CMC/TPF

In [6]:
tpf = TensorPatchFitter(backend, n_shots=n_shots)
tpf.build()

In [7]:
t_res = tpf.apply(res_d)
print(t_res[targ_bv_str])

0.9880767543770003


## What if circuit filter + backend filter?

In [8]:
tj_res = tpf(res_j)
print(tj_res[targ_bv_str])

0.9987656293373195


In [9]:
tpf_filter = FakeMeasurementError(n_qubits=4, meas_filter=tpf)
jt_res = jigsaw.jigsaw(circuit, backend, n_shots, verbose=True, local_pmf_pairs = [(0, 3), (1, 2)], probs=tpf_filter)
print(res_j[targ_bv_str])

0.9883356984008368


/home/alan/aur/miniconda3/lib/python3.9/site-packages/PatchedMeasCal-0.2-py3.9.egg/PatchedMeasCal/jigsaw.py:140: RuntimeWarning: invalid value encountered in double_scalars


## Breaking JIGSAW via Normalisation


In [23]:
n_qubits=4
circuit = integer_state_prep_sim(0, n_qubits)
circuit.draw()

# Target state is '0100' as Qiskit reports measured qubits backwards

┌─┐         
q_0: ┤M├─────────
     └╥┘┌─┐      
q_1: ─╫─┤M├──────
      ║ └╥┘┌─┐   
q_2: ─╫──╫─┤M├───
      ║  ║ └╥┘┌─┐
q_3: ─╫──╫──╫─┤M├
      ║  ║  ║ └╥┘
c: 4/═╩══╩══╩══╩═
      0  1  2  3

In [24]:
n_qubits = 4

aligned_cmap = [[0, 3], [2, 1]]
anti_aligned_cmap = [[0, 1], [2, 3]]

backend = LocalSimulator(n_qubits)
probs = FakeMeasurementError(
        [100, 0, 10,0,0], # Const - Controls correlation of error weights
        [0, 0 , 0, 0, 0], # 1 -> 0 - Controls error biases
        [0, 0, 0, 0, 0], # 0 -> 1 - Controls error biases
        n_qubits=n_qubits,
        coupling_map = aligned_cmap,
        norm_error=0.75,
        )

### Aligned Jigsaw

In [25]:
# jigsaw pairs align with the errors
jigsaw.jigsaw(circuit, backend, 
              100, n_qubits=n_qubits, probs=probs, 
              local_pmf_pairs=aligned_cmap)

AttributeError: 'function' object has no attribute 'jigsaw'

### Anti-Aligned Jigsaw

In [26]:
# jigsaw pairs anti_align with the errors
jigsaw(circuit, backend, 
              100, n_qubits=n_qubits, probs=probs, 
              local_pmf_pairs=anti_aligned_cmap)

{'0000': 0.810546875, '0110': 0.095703125, '1001': 0.09375}
[{'00': 1.0}, {'00': 1.0}]
{'0000': 0.3333333333333333, '0110': 0.3333333333333333, '1001': 0.3333333333333333}
{'0000': 0.3333333333333333, '1001': 0.3333333333333333, '0110': 0.3333333333333333}


NameError: name 'rev_results_dict' is not defined

### Aligned CMC

In [14]:
res_d = probs(qiskit.execute(circuit, backend, shots=16000, optimization_level=0, initial_layout=initial_layout).result().get_counts())
tpf = TensorPatchFitter(backend, n_shots=16000, coupling_map=aligned_cmap)
tpf.build(probs=probs)
print('Bare:', res_d)
print('TPF:', tpf.apply(tpf.apply(res_d)))

Bare: {'0010': 1559, '0100': 12831, '1101': 1610}
TPF: {'0100': 16000.0, '0010': 0.0}


### Anti-aligned CMC

In [15]:
# CMC anti-aligned -> I
res_d = probs(qiskit.execute(circuit, backend, shots=16000, optimization_level=0, initial_layout=initial_layout).result().get_counts())
tpf = TensorPatchFitter(backend, n_shots=16000, coupling_map=anti_aligned_cmap)
tpf.build(probs=probs)
print('Bare:', res_d)
print('TPF:', tpf.apply(tpf.apply(res_d)))

Bare: {'0010': 1578, '0100': 12864, '1101': 1558}
TPF: {'0100': 12864.0, '0010': 1578.0, '1101': 1558.0}


In [18]:
from PatchedMeasCal.jigsaw import build_local_pmf_circuit, build_local_pmf_tables, build_global_pmf, convolve

def jigsaw(circuit, backend, n_shots, 
    verbose=False,  # Verbosity
    equal_shot_distribution=False, # Splits number of shots equally, otherwise uses that number of shots per experiment
    local_pmf_pairs=None, # Local pairs, random if not set
    probs=None, # False error prob distribution
    n_qubits=None,  # Number of qubits
    meas_filter=None, # Inbuilt meas filter pass
    norm_fix=False # Our normalisation fix for JIGSAW
    ):

    if n_qubits is None and backend.properties() is not None:
        n_qubits = len(backend.properties()._qubits)

    global_pmf_table = build_global_pmf(circuit, backend, n_shots, probs=probs, n_qubits=n_qubits)

    # If no allocation is provided, use random
    if local_pmf_pairs is None:
        local_pmf_pairs = list(range(n_qubits))
        random.shuffle(local_pmf_pairs)
        local_pmf_pairs = [
                [i, j] for i, j in zip(
                    local_pmf_pairs[::2],
                    local_pmf_pairs[1::2]
                    )
            ]

    if equal_shot_distribution:
        n_shots_global = n_shots // 2
        n_shots_pmfs = n_shots // (2 * len(local_pmf_pairs))
    else:
        n_shots_global = n_shots
        n_shots_pmfs = n_shots

    # Because qiskit stores results strings backwards, the index ordering is reversed
    local_pmf_pairs_index = [[(n_qubits - i - 1), (n_qubits - j - 1)] for i, j in local_pmf_pairs]

    local_pmf_circs = [build_local_pmf_circuit(circuit, backend, pairs, n_qubits=n_qubits) for pairs in local_pmf_pairs]

    local_pmf_tables = build_local_pmf_tables(
        local_pmf_circs,
        local_pmf_pairs_index,
        backend,
        n_shots_pmfs,
        probs=probs,
        n_qubits=n_qubits)
    print(global_pmf_table)
    print(local_pmf_tables)
    for table, pair in zip(local_pmf_tables, local_pmf_pairs):
        global_pmf_table = convolve(global_pmf_table, table, pair, norm_fix=norm_fix)
        print(global_pmf_table)

    global_pmf_table = rev_results_dict(global_pmf_table)
    return global_pmf_table